In [ ]:
from zipfile import ZipFile

with ZipFile('cell_images.zip', 'r') as zip:
  zip.extractall()

In [ ]:
import pandas as pd
import numpy as np

from keras import regularizers
from PIL import Image
import os
import cv2
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
            
from random import shuffle
from skimage.transform import resize
from tensorflow.keras.utils import to_categorical

In [ ]:
data=[]
labels=[]
Parasitized=os.listdir("malaria/cell_images/Parasitized/")
for a in Parasitized:
    try:
        image=cv2.imread("malaria/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

In [ ]:
Uninfected=os.listdir("malaria/cell_images/Uninfected/")
for b in Uninfected:
    try:
        image=cv2.imread("malaria/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")

In [ ]:
Cells=np.array(data)
labesl=np.array(labels)

In [ ]:
np.save("Cells",Cells)
np.save("labels",labels)

In [ ]:
Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

In [ ]:
s=np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [ ]:
num_classes=len(np.unique(labels))
len_data=len(Cells)

In [ ]:
(x_train,x_test)=Cells[(int)(0.1*len_data):],Cells[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

In [ ]:
(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

In [ ]:
y_train=to_categorical(y_train,num_classes)
y_test=to_categorical(y_test,num_classes)

Data Agumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False)  

In [ ]:
datagen.fit(x_train)

Early Stopping

In [ ]:
from keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor='val_accuracy', patience=3)
epochs = 20 # 
batch_size = 256

Neural network

In [ ]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

Dropout 1

In [ ]:
model.add(Dropout(0.2))
model.add(Flatten())

L2 Regularizer

In [ ]:
model.add(Dense(500,kernel_regularizer=regularizers.l2(0.01),activation="relu"))

Dropout 2

In [ ]:
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 50, 50, 16)        208       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 25, 25, 32)        2080      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 64)        8256      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 6, 64)        

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[earlystop])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
96/96 [==============================] - 62s 641ms/step - loss: 1.8465 - accuracy: 0.6201 - val_loss: 0.6246 - val_accuracy: 0.6864
Epoch 2/20
96/96 [==============================] - 62s 644ms/step - loss: 0.6146 - accuracy: 0.6927 - val_loss: 0.5627 - val_accuracy: 0.7441
Epoch 3/20
96/96 [==============================] - 60s 628ms/step - loss: 0.5153 - accuracy: 0.7889 - val_loss: 0.4146 - val_accuracy: 0.8838
Epoch 4/20
96/96 [==============================] - 62s 642ms/step - loss: 0.3652 - accuracy: 0.8846 - val_loss: 0.2619 - val_accuracy: 0.9365
Epoch 5/20
96/96 [==============================] - 60s 628ms/step - loss: 0.2955 - accuracy: 0.9119 - val_loss: 0.2420 - val_accuracy: 0.9252
Epoch 6/20
96/96 [==============================] - 60s 626ms/step - loss: 0.2774 - accuracy: 0.9204 - val_loss: 0.2126 - val_accuracy: 0.9448
Epoch 7/20
96/96 [==============================] - 62s 639ms/step - loss: 0.2538 - accuracy: 0.9295 - val_loss: 0.2125 - val_accuracy: 0.9441

In [ ]:
pred = model.predict(x_test)

In [ ]:
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print("Train Accuracy:", train_acc*100)
print("Test Accuracy:", test_acc*100)

Train Accuracy: 95.44410109519958
Test Accuracy: 95.9709644317627
